analyze NCI cervical cnv results

In [69]:
# import csv
import matplotlib
# import seaborn
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scipy.stats import ttest_ind
# import scipy.stats as stats
# import scipy.stats as ss
# from matplotlib import style
# import matplotlib.gridspec as gridspec
# import matplotlib.patches as patches
matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (6, 4)
from adjustText import adjust_text
import sys
sys.path.append('/projects/trans_scratch/validations/workspace/szong/Cervical/notebook')
import sz_toolbox as szt


# from sklearn.decomposition import RandomizedPCA, PCA, TruncatedSVD
# from sklearn.linear_model import LogisticRegression
# from sklearn.manifold import TSNE
# from sklearn.preprocessing import StandardScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

reference paper: https://www.sciencedirect.com/science/article/pii/S1535610819300960#fig1

definition of gain, amplification, loss, and deletion:

(C) Known cancer-related genes located in significant GISTIC peaks with residual q < 1 × 10−4 (amplification, gain, neutral, loss, and deletion were defined as GISTIC +2, +1, 0, −1, and −2, respectively).

(A–C) Asterisks indicate associations with mRNA subtypes (intrinsic subtype, ASCAT ploidy, lymph node status, and somatic copy-number alterations were tested using Pearson's chi-square test; ASCAT purity and homologous recombination deficiency [HRD] score were tested using the Kruskal-Wallis test; age at surgery was tested using analysis of variance; apocrine differentiation and somatic mutations were tested using Fisher's exact test. ∗∗p < 0.01, ∗p < 0.05).

To assess somatic copy number alterations (CNAs), reported oncogenes and tumor suppressor genes observed in the top rank (residual q < 1 × 10−4) GISTIC peaks were examined 

 GISTIC2.0 was run with the following parameters changed from the default settings (-ta 0.2 -td 0.2 -genegistic 1 -smallmem 1 -broad 1 -conf 0.95 -rx 0 –brlen 0.7 -cap 3.5 –armpeel 1).

### Starting here 20190401

* look at only cytoband level at this point, do not break down to individual gene. 
* verify if the markerfile used is ok. They differ in number of probes for 3 separate runs: hiv+ /-/tcga. should i use the same markerfile for these 3 runs.
* I believe the markerfile at least affect the segment count, which resulted in the elimination of 2 patients in the run including all 118 patients.
* i think i will need to use the same marker file for all three runs. The markers file identifies the marker positions used in the original dataset (before segmentation) for array or capture experiments. This is not just the segment start and end position.
* tcga used a slightly different marker file. snp6 + tcga start/end positions

In [119]:
root = '/projects/trans_scratch/validations/workspace/szong/Cervical/cnv/gatk_cnv/'
# outputdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/cnv/gatk_cnv/shared_uniq_genes/'
oncogenef = '/home/szong/projects/resource/KB_280_oncogenes_and_tumour_suppressors.txt'
gistdir = '/home/szong/bin/GISTIC_2_0_22/'
runs = ['gatk_118_HIV_pos', 'gatk_118_HIV_neg', 'tcga']
alterations = ['amp', 'del']

In [110]:
# get list of oncogenes and tumour suppressors
tgs = pd.read_csv(oncogenef, header=None)[0].tolist()
tgs[:2]
len(tgs)

['A1CF', 'ABI1']

778

### 1. reformat gistic output

In [111]:
def summarize_genes(inf, tgs):
    df = pd.read_csv(inf, sep='\t', header=None)#.replace('', 'None')
#     file_name = inf.split('/')[-1]
#     print(df)
    df.loc['genes',:] = df.apply(lambda x:  ','.join([i.strip() for i in set(x[~x.isnull()][4:]) if ('[' not in i)]))
    df.loc['tg_genes',:] = df.apply(lambda x:  ','.join([m.strip() for m in set(x[~x.isnull()][4:]) if m in tgs]))
    df1 = df.loc[[0,1,2,3,'genes', 'tg_genes'],:].T
    df1['xxx'] = df1.genes.apply(lambda x: x.split(','))
#     '' in the list when there is a blank in the original files, must have a better way to deal with this
    df1['num_genes'] = df1['xxx'].apply(lambda x: len(x) if '' not in x else len(x)-1)
    df1 = df1.drop('xxx', axis=1)
    df1 = df1.drop(0)
    df1 = df1[:-1]
    df1.columns = ['cytoband', 'q_value', 'residual_q_value', 'wide_peak_boundary', 'genes_in_wide_peak','oncogenes','num_genes']
#     df1['cancer_genes'] = df1.apply(lambda x: '({0}){1}'.format(x[4], x[5]), axis=1)
    of = f'{inf}.summary'
    print('write summary to: ', of)
    df1.to_csv(of, index=False, sep='\t')
    return df1

In [112]:
def get_qvalue_cytoband(f):
    df = pd.read_csv(f, sep='\t')
    df.columns = ['cytoband', 'q_value', 'residual_q_value', 'chrom', 'wide_peak_boundary', 'genes_in_wide_peak','oncogenes','num_genes']
#     print(df.head(2), df.shape, df.columns)
    df['gene'] = df['genes_in_wide_peak']#.apply(lambda x: x.split(',')[0])
    df = df[['q_value','cytoband', 'gene']]
    df = df[df.gene != '']
    s = df["gene"].str.split(',', expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = df.loc[i].copy()
    df2["gene"] = s.values
    return df2

In [120]:
for alter in alterations:
    for run in runs:
#         print(f'{gistdir}{run}/{alter}_genes.conf_99.txt')
        f = f'{gistdir}{run}/{alter}_genes.conf_99.txt'
        df = summarize_genes(f, tgs)
#         df.shape
#         break

write summary to:  /home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_pos/amp_genes.conf_99.txt.summary
write summary to:  /home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_neg/amp_genes.conf_99.txt.summary
write summary to:  /home/szong/bin/GISTIC_2_0_22/tcga/amp_genes.conf_99.txt.summary
write summary to:  /home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_pos/del_genes.conf_99.txt.summary
write summary to:  /home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_neg/del_genes.conf_99.txt.summary
write summary to:  /home/szong/bin/GISTIC_2_0_22/tcga/del_genes.conf_99.txt.summary


### 2. parse conf_99 files

In [115]:
def get_cytobands(f):
    df = pd.read_csv(f, sep='\t')
    df = df.iloc[:,[0,1,5,6]]
    df.columns = ['cytoband', 'q_value', 'oncogenes', 'num_genes']
    return df

In [121]:
# HIV +
pos_amp_file = '/home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_pos/amp_genes.conf_99.txt.summary'
pos_del_file = '/home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_pos/del_genes.conf_99.txt.summary'
pos_amps = get_cytobands(pos_amp_file)
pos_amps.head(2)
pos_dels = get_cytobands(pos_del_file)
pos_dels.head(2)

,cytoband,q_value,oncogenes,num_genes
0,11q22.2,6.734800e-09,"BIRC3,YAP1",7
1,8q24.21,7.315600e-04,NaN,0


,cytoband,q_value,oncogenes,num_genes
0,2q22.1,9.596400e-16,LRP1B,1
1,19p13.3,1.422000e-10,STK11,3


In [122]:
# HIV -
neg_amp_file = '/home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_neg/amp_genes.conf_99.txt.summary'
neg_del_file = '/home/szong/bin/GISTIC_2_0_22/gatk_118_HIV_neg/del_genes.conf_99.txt.summary'
neg_amps = get_cytobands(neg_amp_file)
neg_amps.head(2)
neg_dels = get_cytobands(neg_del_file)
neg_dels.head(2)

,cytoband,q_value,oncogenes,num_genes
0,11q22.1,0.000573,"BIRC3,YAP1",21
1,3q28,0.000039,"MECOM,IGF2BP2,SOX2,MUC4,KLHL6,MAP3K13,PRKCI,PI...",234


,cytoband,q_value,oncogenes,num_genes
0,21p11.2,5.576100e-09,NaN,16
1,9p24.1,1.789600e-07,"CD274,PDCD1LG2",4


In [123]:
# TCGA
tcga_amp_file = '/home/szong/bin/GISTIC_2_0_22/tcga/amp_genes.conf_99.txt.summary'
tcga_del_file = '/home/szong/bin/GISTIC_2_0_22/tcga/del_genes.conf_99.txt.summary'
tcga_amps = get_cytobands(tcga_amp_file)
tcga_amps.head(2)
tcga_dels = get_cytobands(tcga_del_file)
tcga_dels.head(2)

,cytoband,q_value,oncogenes,num_genes
0,11q22.1,1.389600e-25,"BIRC3,YAP1",7
1,3q26.31,1.183600e-13,NaN,3


,cytoband,q_value,oncogenes,num_genes
0,2q37.1,1.683100e-25,"CUL3,PAX3,HDAC4,IRS1",191
1,11q25,2.064600e-15,"KCNJ5,ETS1,PAFAH1B2,CHEK1,CBL,FOXR1,PCSK7,DDX6...",201


### 2. summarize cytobands

In [124]:
def merge_qvalues(df1, df2):
    df = pd.merge(df1, df2, left_on='cytoband', right_on='cytoband', how='outer')#.fillna(1)
    new_cols = pd.Series(df.columns.tolist()).str.replace('_x', '_hiv+').str.replace('_y', '_hiv-')
    df.columns = new_cols
    df = df.drop_duplicates(subset='cytoband', keep='last')
    df['cytoband'].nunique()
    df['q_value_hiv+'] = df['q_value_hiv+'].fillna(1)
    df['q_value_hiv-'] = df['q_value_hiv-'].fillna(1)
    df['hiv+q(-log10)'] = -np.log10(df['q_value_hiv+'])
    df['hiv-q(-log10)'] = -np.log10(df['q_value_hiv-'])
    # df['num_genes_hiv+'] = df['num_genes_hiv+'].fillna(0).astype(int)
    # df['num_genes_hiv-'] = df['num_genes_hiv-'].fillna(0).astype(int)
    # df['oncogenes_hiv+'] = df['oncogenes_hiv+'].fillna('None')
    # df['oncogenes_hiv-'] = df['oncogenes_hiv-'].fillna('None')
    return df

In [125]:
htmcp_amps = merge_qvalues(pos_amps, neg_amps)
cnvdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/cnv/'
of = f'{cnvdir}amps_cytoband_qvalue_comparison_hiv_pos_vs_neg.csv'
htmcp_amps.to_csv(of, index=False)

htmcp_dels = merge_qvalues(pos_dels, neg_dels)

of = f'{cnvdir}dels_cytoband_qvalue_comparison_hiv_pos_vs_neg.csv'
htmcp_dels.to_csv(of, index=False)

tcga_amps = merge_qvalues(tcga_amps, neg_amps)

of = f'{cnvdir}amps_cytoband_qvalue_comparison_tcga_vs_neg.csv'
tcga_amps.to_csv(of, index=False)

tcga_dels = merge_qvalues(tcga_dels, neg_dels)

of = f'{cnvdir}dels_cytoband_qvalue_comparison_tcga_vs_neg.csv'
tcga_dels.to_csv(of, index=False)

In [107]:
htmcp_amps.head(2)

,cytoband,q_value_hiv+,oncogenes_hiv+,num_genes_hiv+,q_value_hiv-,oncogenes_hiv-,num_genes_hiv-,hiv+q(-log10),hiv-q(-log10)
0,11q22.1,3.544700e-09,"BIRC3,YAP1",7.0,0.001928,"BIRC3,YAP1",21.0,8.450421,2.714915
1,8q24.21,1.045400e-04,NaN,0.0,1.000000,NaN,NaN,3.980718,-0.000000
